In [64]:
import pandas as pd
from sklearn.datasets import load_iris
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rnna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

In [65]:
##[rgdpna]is GDP（国民計算）
##[rkna資本サービスis Capital services 蓄積された非金融資産が生産活動に与える貢献（フロー）を表す概念
##[pop]is population
##[emp]就業者数number of persons engaged 
##[avh]年間平均労働時間Average annual hours worked by persons engaged 
##[labsh]現在の国内価格におけるGDPに占める労働報酬の割合
##       Share of labour compensation in GDP at current national prices
##[rtfpna]TFP at constant national prices (2017=1)
##        Reports total factor productivity (TFP) levels at constant prices against the reference year
##        基準年に対する一定価格での全要素生産性（TFP）レベル
##        TEP 全要素生産性→資本と労働の増加によらない生産の増加を表すもの

In [66]:
# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # 対数化　GDP per worker
data['k_pc'] = np.log(data['rnna'] / data['emp'])  #対数化　 Capital per worker
data['alpha'] = 1 - data['labsh'] #資本のシェア


data['y_n'] = data['rgdpna'] / data['emp']  # Y/N(一人当たりGDP)
data['hours'] = data['emp'] * data['avh']  # L（一人当たり労働時間）
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha']))  # A^(1/(1-alpha))　A/A (技術進歩TEPgrowth)
data['cap_term'] = (data['rnna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))  # (K/Y)^(alpha/(1-alpha)) 資本深化capital deepening
data['lab_term'] = data['hours'] / data['pop']  # L/N




In [67]:
# Order by year sort_values→降順
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita


# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean','alpha': 'mean','tfp_term':'mean','cap_term':'mean','labsh':'mean'})




# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['TEP Growth'] = summary['tfp_term'] 
summary['Capital Deepening'] = summary['cap_term']
summary['TEP Share'] = summary['labsh']
summary['Capital Share'] = summary['alpha']

remove = ["g","alpha","tfp_term","cap_term","labsh"]
summary_1 = summary.drop(remove,axis=1)
# Print output
print(summary_1)

             Growth Rate  TEP Growth  Capital Deepening  TEP Share  \
countrycode                                                          
CAN             0.887700    0.994532           2.120823   0.653810   
DEU             0.661055    0.925765           2.462558   0.629270   
FRA             0.829236    1.012616           2.838994   0.618806   
GBR             1.015544    0.993804           2.946920   0.587007   
ITA            -0.037181    1.169158           5.706703   0.515165   
JPN             0.710956    0.955232           3.276744   0.569399   
USA             1.536985    0.928927           2.271405   0.606740   

             Capital Share  
countrycode                 
CAN               0.346190  
DEU               0.370730  
FRA               0.381194  
GBR               0.412993  
ITA               0.484835  
JPN               0.430601  
USA               0.393260  


In [68]:
summary_1

,Growth Rate,TEP Growth,Capital Deepening,TEP Share,Capital Share
countrycode,,,,,
CAN,0.887700,0.994532,2.120823,0.653810,0.346190
DEU,0.661055,0.925765,2.462558,0.629270,0.370730
FRA,0.829236,1.012616,2.838994,0.618806,0.381194
GBR,1.015544,0.993804,2.946920,0.587007,0.412993
ITA,-0.037181,1.169158,5.706703,0.515165,0.484835
JPN,0.710956,0.955232,3.276744,0.569399,0.430601
USA,1.536985,0.928927,2.271405,0.606740,0.393260
